In [11]:
# Download in terminal: 
# pip install fix_yahoo_finance --upgrade --no-cache-dir
import pandas as pd
import matplotlib.pyplot as plt
from pandas_datareader import data as pdr
import fix_yahoo_finance as yf 
# Download zipline if not already installed
# pip install zipline
import zipfile

In [9]:
# doanload data from yahoo finanance 
yf.pdr_override()
# Download information for various stocks
AAPL = pdr.get_data_yahoo('AAPL',start = '2009-01-01', end='2017-12-31',
                         interval = '1mo')
AAPL.dropna()

AMZN = pdr.get_data_yahoo('AMZN',start = '2009-01-01', end='2017-12-31',
                         interval = '1mo')
AMZN.dropna()

BA = pdr.get_data_yahoo('BA',start = '2009-01-01', end='2017-12-31',
                         interval = '1mo')
BA.dropna()

FB = pdr.get_data_yahoo('FB',start = '2009-01-01', end='2017-12-31',
                         interval = '1mo')
FB.dropna()

GOOG = pdr.get_data_yahoo('GOOG',start = '2009-01-01', end='2017-12-31',
                         interval = '1mo')
GOOG.dropna()

MA = pdr.get_data_yahoo('MA',start = '2009-01-01', end='2017-12-31',
                         interval = '1mo')
MA.dropna()

MSFT = pdr.get_data_yahoo('MSFT',start = '2009-01-01', end='2017-12-31',
                         interval = '1mo')
MSFT.dropna()

NVDA = pdr.get_data_yahoo('NVDA',start = '2009-01-01', end='2017-12-31',
                         interval = '1mo')
NVDA.dropna()

UNH = pdr.get_data_yahoo('UNH',start = '2009-01-01', end='2017-12-31',
                         interval = '1mo')
UNH.dropna()

V = pdr.get_data_yahoo('V',start = '2009-01-01', end='2017-12-31',
                         interval = '1mo')
V.dropna()

# Making the data more concise 




[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2009-01-01,13.23,14.49,10.44,12.34,9.93,6.424464e+08
2009-02-01,12.06,14.49,11.89,14.18,11.41,7.455432e+08
2009-03-01,13.98,14.24,12.19,13.90,11.28,9.277952e+08
2009-04-01,13.69,17.00,13.41,16.24,13.17,8.916980e+08
2009-05-01,16.04,17.14,15.66,16.93,13.73,7.946888e+08
2009-06-01,17.12,17.81,15.07,15.56,12.71,7.722140e+08
2009-07-01,15.57,17.42,14.50,16.36,13.36,6.149908e+08
2009-08-01,16.55,17.83,16.48,17.77,14.51,3.759148e+08
2009-09-01,17.69,18.75,17.15,17.28,14.19,3.962068e+08
